In [64]:
import torch

anomaly_maps = torch.randn(40, 1, 256, 256)  # Example anomaly maps tensor
anomaly_scores = torch.randn(40)  # Example anomaly scores tensor

num_tasks = 10

anomaly_maps = anomaly_maps.view(-1, num_tasks, anomaly_maps.size(1), anomaly_maps.size(2), anomaly_maps.size(3))
anomaly_scores = anomaly_scores.view(-1, num_tasks) 

print(anomaly_maps.shape, anomaly_scores.shape)

torch.Size([4, 10, 1, 256, 256]) torch.Size([4, 10])


In [65]:
max_scores = anomaly_scores.argmax(dim=1)
batch_idx = torch.arange(anomaly_scores.size(0))

In [66]:
max_anomaly_maps = anomaly_maps[batch_idx, max_scores]
max_scores = anomaly_scores[batch_idx, max_scores]

In [67]:
max_anomaly_maps.shape

torch.Size([4, 1, 256, 256])

In [68]:
max_scores.shape

torch.Size([4])

In [1]:
from moviad.utilities.custom_feature_extractor_trimmed import CustomFeatureExtractor
from moviad.models.cfa import CFA, CFATrainArgs
from moviad.trainers.trainer import Trainer
from moviad.datasets.mvtec import MVTecDataset
from torch.utils.data import Subset
from moviad.datasets.dataset_arguments import DatasetArguments
from moviad.utilities.evaluation.metrics import MetricLvl, RocAuc, AvgPrec, F1, ProAuc
import torch
import wandb


feature_extractor = CustomFeatureExtractor("wide_resnet50_2", ["layer1", "layer2", "layer3"], frozen=True)    

args = DatasetArguments(
    dataset_path = "/mnt/mydisk/manuel_barusco/datasets/mvtec",
    img_size = (256, 256),
    gt_mask_size = (256, 256),
    image_transform_list = None
)

train_dataset = MVTecDataset(args, category="bottle", split="train")
train_dataset = Subset(train_dataset, list(range(0, 10)))  # use a subset for faster testing
test_dataset = MVTecDataset(args, category="bottle", split="test")

model = CFA(feature_extractor, "wide_resnet50_2")

/home/manuel_barusco/moviad/moviad/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model.parameters()

<generator object Module.parameters at 0x7954100d9000>

In [3]:
model

CFA(
  (Descriptor): Descriptor(
    (layer): CoordConv2d(
      1792, 1792, kernel_size=(1, 1), stride=(1, 1)
      (addcoords): AddCoords()
      (conv): Conv2d(1794, 1792, kernel_size=(1, 1), stride=(1, 1))
    )
  )
)